In [44]:
def read_input_file(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()

In [45]:
!pip install rouge-score

In [46]:
from transformers import GPT2Tokenizer,GPT2LMHeadModel
# Instantiating the model and tokenizer with gpt
tokenizer = GPT2Tokenizer.from_pretrained('microsoft/BioGPT-Large')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = GPT2LMHeadModel.from_pretrained('microsoft/BioGPT-Large')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BioGptTokenizer'. 
The class this function is called from is 'GPT2Tokenizer'.
You are using a model of type biogpt to instantiate a model of type gpt2. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at microsoft/BioGPT-Large were not used when initializing GPT2LMHeadModel: ['biogpt.layers.32.self_attn.out_proj.weight', 'biogpt.layers.47.self_attn.v_proj.weight', 'biogpt.layers.21.fc1.bias', 'biogpt.layers.5.self_attn.k_proj.bias', 'biogpt.layers.38.self_attn.q_proj.bias', 'biogpt.layers.30.self_attn.v_proj.weight', 'biogpt.layers.5.self_attn.q_proj.bias', 'biogpt.layers.37.fc1.bias', 'biogpt.layers.40.self_attn.out_proj.bias', 'biogpt.layer_norm.weight', 'biogpt.layers.6.self_attn.q_proj.weight', 'biogpt.layers.2

In [47]:
file_path = "/kaggle/input/full-texts/FullText-2529.txt"
input_text = read_input_file(file_path)
input_text = input_text

In [48]:
inputs = tokenizer.encode_plus(input_text, return_tensors='pt', truncation=False, padding='longest')
input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']

# pad the input_ids and attention_mask with the new padding token
max_length = 512
padded_input_ids = input_ids[:, :max_length].reshape(-1, max_length)
padded_attention_mask = attention_mask[:, :max_length].reshape(-1, max_length)

summary_ids = model.generate(padded_input_ids, attention_mask=padded_attention_mask, early_stopping=False, min_length=90, max_length=max_length, pad_token_id=tokenizer.eos_token_id)

GPT_summary=tokenizer.decode(summary_ids[0],skip_special_tokens=True)
print(GPT_summary)

Token indices sequence length is longer than the specified maximum sequence length for this model (5224 > 1024). Running this sequence through the model will result in indexing errors
Input length of input_ids is 512, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Serum immunoglobulin G4-related sclerosing cholangitis IgG4-SC is a type of autoimmune pancreatitis associated with elevated serum IgG4 levels. None,NoneTypes 3 and 4 IgG4-SC are difficult to differentiate from bile duct cancer, and hepatectomy has been reported sporadically in such situations. None,NoneWe describe a rare case of a patient with IgG4-SC but without pancreatic lesions. Accurate diagnosis was made, without the need for performing a partial hepatectomy. The patient was a 69-year-old man referred for the evaluation of jaundice and steatorrhea persisting for 1 week. He had a prior history of hypertension, diabetes mellitus, and benign prostatic hypertrophy, with no previous pancreatic disease or autoimmune diseases. Physical examination revealed normal vital signs, scleral icterus, and jaundice, with no abnormalities on abdominal examination. Laboratory studies showed elevated total bilirubin, liver, and biliary enzyme levels. Endoscopic retrograde cholangiopancreatography w

In [49]:
from rouge_score import rouge_scorer

def compute_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return scores

if __name__ == '__main__':
    file_path = "/kaggle/input/full-texts/Abstract-2529.txt"
    input_abstract = read_input_file(file_path)
    reference =  GPT_summary 
    candidate = input_abstract

    scores = compute_rouge(reference, candidate)
    for key, score in scores.items():
        print(f"{key.upper()}:")
        print(f"  Precision: {score.precision:.4f}")
        print(f"  Recall: {score.recall:.4f}")
        print(f"  F1 Score: {score.fmeasure:.4f}\n")


ROUGE1:
  Precision: 0.4606
  Recall: 0.4419
  F1 Score: 0.4510

ROUGE2:
  Precision: 0.1585
  Recall: 0.1520
  F1 Score: 0.1552

ROUGEL:
  Precision: 0.1818
  Recall: 0.1744
  F1 Score: 0.1780

ROUGELSUM:
  Precision: 0.1818
  Recall: 0.1744
  F1 Score: 0.1780

